In [ ]:
!pip install scikit-multiflow

     |████████████████████████████████| 1.1 MB 7.6 MB/s 


In [ ]:
# multi-class classification with Keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
 
# load dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
 
# Loading a dataset

#url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataBM_bcp.csv'
#url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataGM_bcp.csv'
#url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataBM_gender.csv'
#url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataGM_gender.csv'
#url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataBM_sm.csv'
url = '/content/drive/MyDrive/Dataset/Dataset_without0/FinalDataGM_sm.csv'
 
data_frame = pd.read_csv(url, delimiter=';')
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~data_frame.isin([np.nan, np.inf, -np.inf]).any(1)
    return data_frame[indices_to_keep].astype(np.float64)
 
data_frame = clean_dataset(data_frame)
# Splitting the dataset
#X, Y = data_frame.drop(columns="bcp"), data_frame.bcp
#X, Y = data_frame.drop(columns="gender"), data_frame.gender
X, Y = data_frame.drop(columns="smoker"), data_frame.smoker
 
train_X, test_X, train_Y, test_Y = train_test_split(
    X, Y, test_size=0.2, random_state=42
)
 
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_Y)
encoded_Y = encoder.transform(train_Y)
 
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(150, input_dim=1589, activation='relu'))
    model.add(Dense(50, activation='softmax'))
    model.add(Dense(150, input_dim=1589, activation='relu'))
    model.add(Dense(80, activation='softmax'))
    model.add(Dense(33, activation='softmax'))
    model.add(Dense(4, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
 
estimator = KerasClassifier(build_fn=baseline_model, epochs=3000, batch_size=5, verbose=True)
train_model = estimator.fit(train_X, train_Y)
predicted = estimator.predict(test_X)

Výstupní stream byl oříznut na posledních 5000 řádků.
72/72 [==============================] - 0s 3ms/step - loss: 1.0179 - accuracy: 0.6417
Epoch 502/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0180 - accuracy: 0.6417
Epoch 503/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0177 - accuracy: 0.6417
Epoch 504/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0177 - accuracy: 0.6417
Epoch 505/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0176 - accuracy: 0.6417
Epoch 506/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0177 - accuracy: 0.6417
Epoch 507/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0177 - accuracy: 0.6417
Epoch 508/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0176 - accuracy: 0.6417
Epoch 509/3000
72/72 [==============================] - 0s 3ms/step - loss: 1.0174 - accuracy: 0.6417
Epoch 510/3000
72/72 [=====================

In [ ]:
#DDM
from skmultiflow.drift_detection.ddm import DDM
ddm = DDM()

# Adding stream elements to DDM and verifying if drift occurred

for i in range(len(predicted)):
    ddm.add_element(predicted[i])
    if ddm.detected_warning_zone():
        print('Warning zone has been detected in data: ' + str(predicted[i]) + ' - of index: ' + str(i))
    if ddm.detected_change():
        print('Change detected in data: ' + str(predicted[i]) + ' - at index: ' + str(i))


/usr/local/lib/python3.7/dist-packages/skmultiflow/drift_detection/ddm.py:126: RuntimeWarning: invalid value encountered in sqrt
  self.miss_std = np.sqrt(self.miss_prob * (1 - self.miss_prob) / float(self.sample_count))


In [ ]:
#EDDM
from skmultiflow.drift_detection.eddm import EDDM
eddm = EDDM()

# Adding stream elements to EDDM and verifying if drift occurred

for i in range(len(predicted)):
    eddm.add_element(predicted[i])
    if eddm.detected_warning_zone():
        print('Warning zone has been detected in data: ' + str(predicted[i]) + ' - of index: ' + str(i))
    if eddm.detected_change():
        print('Change detected in data: ' + str(predicted[i]) + ' - at index: ' + str(i))